In [5]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')


In [6]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

Handle class imbalance

In [8]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

#### Track Experiments

In [9]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [10]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)


reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669725,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.91326105087573,
   'support': 300.0}},
 {'0': {'precision': 0.9676258992805755,
   'recall': 0.9962962962962963,
   'f1-score': 0.9817518248175182,
   'support': 270.0},
  '1': {'precision': 0.9545454545454546,
   'recall': 0.7,
   'f1-score': 0.8076923076923077,
   'support': 30.0},
  'accuracy': 0.9666666666666667,
  'macro avg': {'precision': 0.961085676913015,
   'recall': 0.8481481481481481,
   'f1-score': 0.8947220662549129,
   'support': 300.0},
  'weighted avg': {'precision': 0.9663

In [11]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost


In [12]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection 2")
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy' : report['accuracy'],
            'recall_class_0' : report['0']['recall'],
            'recall_class_1' : report['1']['recall'],
            'f1_score_macro' : report['macro avg']['f1-score'],
        })

        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")


2025/07/05 14:21:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/05 14:21:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/05 14:21:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/218409269353444867/runs/6ab998032d284350a36f97379b0bcefe
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/218409269353444867


2025/07/05 14:21:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/05 14:21:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/218409269353444867/runs/5ed855f6ac7e44479609d050009ee307
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/218409269353444867


2025/07/05 14:21:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/05 14:21:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/218409269353444867/runs/09982a42ff114af681eb967d28d9aeef
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/218409269353444867


2025/07/05 14:21:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/218409269353444867/runs/3d4626029d784812b905d3c0976a199e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/218409269353444867


### Register the Model


In [13]:
model_name = 'XGB-Smote 2'
run_id= input("Please enter the run ID: ")
model_uri = f"runs:/{run_id}/model"

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)
    print(f"Model {model_name} registered successfully.")


Registered model 'XGB-Smote 2' already exists. Creating a new version of this model...
2025/07/05 14:21:26 WARNING mlflow.tracking._model_registry.fluent: Run with id a4089161115d4f7cb2f840c025e049a0 has no artifacts at artifact path 'model', registering model based on models:/m-7218f93749c54b6cbd6149ad4bbc6a82 instead
2025/07/05 14:21:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote 2, version 1


Model XGB-Smote 2 registered successfully.
🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/218409269353444867/runs/a4089161115d4f7cb2f840c025e049a0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/218409269353444867


Created version '1' of model 'XGB-Smote 2'.


Load the model

In [16]:
import pandas as pd
model_version = 1
model_uri = f"models:/{model_name}@challenger"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [17]:
y_pred_df = pd.DataFrame(y_pred, columns=['Predicted']) 
y_pred_df['Actual'] = y_test
y_pred_df['Correct'] = y_pred_df['Predicted'] == y_pred_df['Actual']
y_pred_df['Correct'] = y_pred_df['Correct'].astype(int)
y_pred_df[:4]

,Predicted,Actual,Correct
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1


### Transition the Model to Production

In [18]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(
    src_model_uri=current_model_uri,
    dst_name= production_model_name
)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote 2' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1751722534253, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1751722534253, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='a4089161115d4f7cb2f840c025e049a0', run_link='', source='models:/XGB-Smote 2/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [27]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

In [28]:
loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])